In [2]:
import sqlite3
import pandas as pd

In [3]:
# Kết nối SQLite 
conn = sqlite3.connect("students.db") 
cursor = conn.cursor()

In [4]:
cursor.execute("DROP TABLE IF EXISTS student;")
cursor.execute("DROP TABLE IF EXISTS course;")

# Tạo bảng student
cursor.execute("""
CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
);
""")

# Tạo bảng course
cursor.execute("""
CREATE TABLE course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
);
""")

# Lưu thay đổi vào database
conn.commit()

In [5]:
# Dữ liệu cho bảng student
student_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2),
    (3, "Pham Van Nam", "Toan Tin", None, 7.9),   
    (4, "Le Thanh Huyen", "Toan Tin", 20, 7.2),
    (5, "Vu Quoc Anh", "May Tinh", 24, 8.0),
    (6, "Dang Thuy Linh", "May Tinh", 24, 5.5),
    (7, "Bui Tien Dung", "Kinh Te", 34, 9.2),
    (8, "Ho Ngoc Mai", "Toan Tin", 20, 8.8),
    (9, "Duong Huu Phuc", "Kinh Te", None, 7.2),   
    (10, "Cao Thi Hanh", "May Tinh", None, 7.0)     
]

# Dữ liệu cho bảng course
course_data = [
    (12, "Giai tich"),
    (34, "Thong ke"),
    (26, "Tin hoc")
]

# Chèn dữ liệu vào bảng student
cursor.executemany("INSERT INTO student VALUES (?, ?, ?, ?, ?);", student_data)

# Chèn dữ liệu vào bảng course
cursor.executemany("INSERT INTO course VALUES (?, ?);", course_data)

conn.commit()

In [6]:
# Đọc dữ liệu từ bảng student
student_df = pd.read_sql_query("SELECT * FROM student;", conn)
print("Bảng Student:")
display(student_df)

# Đọc dữ liệu từ bảng course
course_df = pd.read_sql_query("SELECT * FROM course;", conn)
print("Bảng Course:")
display(course_df)

Bảng Student:


,student_id,name,class,course_id,score
0,1,Nguyen Minh Hoang,May Tinh,12.0,6.7
1,2,Tran Thi Lan,Kinh Te,34.0,9.2
2,3,Pham Van Nam,Toan Tin,NaN,7.9
3,4,Le Thanh Huyen,Toan Tin,20.0,7.2
4,5,Vu Quoc Anh,May Tinh,24.0,8.0
5,6,Dang Thuy Linh,May Tinh,24.0,5.5
6,7,Bui Tien Dung,Kinh Te,34.0,9.2
7,8,Ho Ngoc Mai,Toan Tin,20.0,8.8
8,9,Duong Huu Phuc,Kinh Te,NaN,7.2
9,10,Cao Thi Hanh,May Tinh,NaN,7.0


Bảng Course:


,id,course_name
0,12,Giai tich
1,26,Tin hoc
2,34,Thong ke


trong bảng Student:

Số lượng: 10 dòng, 5 cột.
Bảng Student có các cột: student_id (ID sinh viên, kiểu số nguyên), name (tên sinh viên, kiểu chuỗi), class (lớp, kiểu chuỗi), course_id (ID môn học, kiểu số thực, có giá trị thiếu), score (điểm, kiểu số thực).
Cột course_id có 3 giá trị bị thiếu (NaN) ở các sinh viên có student_id là 3, 9, và 10. Điều này có thể ảnh hưởng đến việc xác định môn học của các sinh viên này.

Trong bảng Course:

Số lượng: 3 dòng, 2 cột
Bảng Course có các cột: id (ID môn học, kiểu số nguyên) và course_name (tên môn học, kiểu chuỗi).

In [7]:
#sử dụng tích Decartes
print("Tích Decartes:")
cursor.execute('''
SELECT * FROM student, course;
''')
df_decartes = pd.DataFrame(cursor.fetchall(), columns=['student_id', 'name', 'class', 'course_id', 'score', 'id', 'course_name'])
print(df_decartes)
print("\n")

Tích Decartes:
    student_id               name     class  course_id  score  id course_name
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  26     Tin hoc
2            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  34    Thong ke
3            2       Tran Thi Lan   Kinh Te       34.0    9.2  12   Giai tich
4            2       Tran Thi Lan   Kinh Te       34.0    9.2  26     Tin hoc
5            2       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
6            3       Pham Van Nam  Toan Tin        NaN    7.9  12   Giai tich
7            3       Pham Van Nam  Toan Tin        NaN    7.9  26     Tin hoc
8            3       Pham Van Nam  Toan Tin        NaN    7.9  34    Thong ke
9            4     Le Thanh Huyen  Toan Tin       20.0    7.2  12   Giai tich
10           4     Le Thanh Huyen  Toan Tin       20.0    7.2  26     Tin hoc
11           4     Le Thanh Huyen  Toan Tin      

Kết luận:

Bảng này là kết quả của phép tích Descartes giữa hai bảng students và courses, dẫn đến việc mỗi sinh viên xuất hiện lặp lại cho mỗi môn học.
Có 10 sinh viên và 3 môn học nên bảng kết quả có 10×3=30 dòng.
Cột course_id có giá trị NaN cho thấy có sinh viên chưa đăng ký môn học nhưng vẫn xuất hiện trong tích Descartes.
course_id và score không khớp hoàn toàn với id của bảng courses, chứng tỏ không phải sinh viên nào cũng đăng ký tất cả các môn.
Cột score lặp lại cho mỗi sinh viên ở mỗi môn, cho thấy dữ liệu chưa được tổng hợp chính xác

In [8]:
print("INNER JOIN:")
cursor.execute('''
SELECT * FROM student INNER JOIN course ON student.course_id = course.id;
''')
df_inner = pd.DataFrame(cursor.fetchall(), columns=['student_id', 'name', 'class', 'course_id', 'score', 'id', 'course_name'])
print(df_inner)
print("\n")

INNER JOIN:
   student_id               name     class  course_id  score  id course_name
0           1  Nguyen Minh Hoang  May Tinh         12    6.7  12   Giai tich
1           2       Tran Thi Lan   Kinh Te         34    9.2  34    Thong ke
2           7      Bui Tien Dung   Kinh Te         34    9.2  34    Thong ke




Bảng chỉ hiển thị các dòng có giá trị khớp giữa course_id của bảng students và id của bảng courses.
Chỉ có 3 sinh viên xuất hiện, cho thấy rất ít sinh viên có đăng ký môn học trùng khớp giữa hai bảng.

LEFT JOIN

In [9]:
print("LEFT JOIN:")
cursor.execute('''
SELECT * FROM student LEFT JOIN course ON student.course_id = course.id;
''')
df_left = pd.DataFrame(cursor.fetchall(), columns=['student_id', 'name', 'class', 'course_id', 'score', 'id', 'course_name'])
print(df_left)
print("\n")

LEFT JOIN:
   student_id               name     class  course_id  score    id course_name
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12.0   Giai tich
1           2       Tran Thi Lan   Kinh Te       34.0    9.2  34.0    Thong ke
2           3       Pham Van Nam  Toan Tin        NaN    7.9   NaN        None
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2   NaN        None
4           5        Vu Quoc Anh  May Tinh       24.0    8.0   NaN        None
5           6     Dang Thuy Linh  May Tinh       24.0    5.5   NaN        None
6           7      Bui Tien Dung   Kinh Te       34.0    9.2  34.0    Thong ke
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8   NaN        None
8           9     Duong Huu Phuc   Kinh Te        NaN    7.2   NaN        None
9          10       Cao Thi Hanh  May Tinh        NaN    7.0   NaN        None




LEFT JOIN giữ lại tất cả các bản ghi từ bảng students và chỉ thêm dữ liệu từ bảng courses khi có giá trị khớp.
Các ô NaN ở cột id và course_name cho thấy có nhiều sinh viên có course_id không khớp với id của bảng courses

RIGHT JOIN

In [10]:
print("RIGHT JOIN:")
cursor.execute('''
SELECT * FROM course LEFT JOIN student ON course.id = student.course_id;
''')
df_right = pd.DataFrame(cursor.fetchall(), columns=['id', 'course_name', 'student_id', 'name', 'class', 'course_id', 'score'])
print(df_right)
print("\n")

RIGHT JOIN:
   id course_name  student_id               name     class  course_id  score
0  12   Giai tich         1.0  Nguyen Minh Hoang  May Tinh       12.0    6.7
1  26     Tin hoc         NaN               None      None        NaN    NaN
2  34    Thong ke         7.0      Bui Tien Dung   Kinh Te       34.0    9.2
3  34    Thong ke         2.0       Tran Thi Lan   Kinh Te       34.0    9.2




RIGHT JOIN giữ lại tất cả các bản ghi từ bảng courses và chỉ thêm dữ liệu từ bảng students khi có giá trị khớp.
Có dòng với course_name là "Tin hoc" nhưng các cột thông tin sinh viên đều là NaN, chứng tỏ không có sinh viên nào đăng ký môn này hoặc dữ liệu đăng ký bị thiếu.

FULL OUTER JOIN (mô phỏng bằng UNION của LEFT JOIN và RIGHT JOIN)



In [11]:
query = '''
SELECT * FROM student LEFT JOIN course 
ON student.course_id = course.id
UNION ALL
SELECT * FROM course LEFT JOIN student 
ON course.id = student.course_id
WHERE student.course_id IS NULL;
'''
df = pd.read_sql_query(query, conn)
print("FULL OUTER JOIN: ", df)

FULL OUTER JOIN:      student_id               name     class  course_id  score    id  \
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12.0   
1            2       Tran Thi Lan   Kinh Te       34.0    9.2  34.0   
2            3       Pham Van Nam  Toan Tin        NaN    7.9   NaN   
3            4     Le Thanh Huyen  Toan Tin       20.0    7.2   NaN   
4            5        Vu Quoc Anh  May Tinh       24.0    8.0   NaN   
5            6     Dang Thuy Linh  May Tinh       24.0    5.5   NaN   
6            7      Bui Tien Dung   Kinh Te       34.0    9.2  34.0   
7            8        Ho Ngoc Mai  Toan Tin       20.0    8.8   NaN   
8            9     Duong Huu Phuc   Kinh Te        NaN    7.2   NaN   
9           10       Cao Thi Hanh  May Tinh        NaN    7.0   NaN   
10          26            Tin hoc      None        NaN    NaN   NaN   

   course_name  
0    Giai tich  
1     Thong ke  
2         None  
3         None  
4         None  
5         None  
6     Thon

FULL OUTER JOIN giữ lại tất cả các bản ghi từ cả hai bảng students và courses, điền NaN khi không có giá trị khớp.
Bảng có dòng với id = 26 và course_name = "Tin hoc" nhưng không có sinh viên nào đăng ký, thể hiện dữ liệu bị thiếu hoặc không có sinh viên nào học môn nào

2. Hãy cập nhật những giá trị course_id còn thiếu trong bảng student bằng câu lệnh SQL, trong đó các giá trị được điền là những giá trị nằm trong bảng course và loại bỏ những bản ghi tham gia những môn học không tồn tại bảng course. Sau đó hãy cho biết:

a. Tổng số sinh viên, điểm trung bình của từng lớp.

b. Tổng số sinh viên, điểm trung bình của từng môn học.

c. Phân loại thi đua theo số điểm của từng môn học biết:

- i. Điểm TB ≥ 9.0: Xuất sắc. - ii. 6.0 ≤ Điểm TB ≤ 8.9: Tốt. - iii. Điểm TB < 6.0: Kém

In [12]:
# Cập nhật course_id còn thiếu bằng giá trị ngẫu nhiên từ bảng course
cursor.execute('''
UPDATE student
SET course_id = (
    SELECT id FROM course ORDER BY RANDOM() LIMIT 1
)
WHERE course_id IS NULL;
''')
conn.commit()

# Loại bỏ các bản ghi có course_id không hợp lệ
cursor.execute('''
DELETE FROM student
WHERE course_id NOT IN (SELECT id FROM course);
''')
conn.commit()

# Kiểm tra lại dữ liệu
cursor.execute('SELECT * FROM student;')
print(pd.DataFrame(cursor.fetchall(), columns=['student_id', 'name', 'class', 'course_id', 'score']))

   student_id               name     class  course_id  score
0           1  Nguyen Minh Hoang  May Tinh         12    6.7
1           2       Tran Thi Lan   Kinh Te         34    9.2
2           3       Pham Van Nam  Toan Tin         12    7.9
3           7      Bui Tien Dung   Kinh Te         34    9.2
4           9     Duong Huu Phuc   Kinh Te         12    7.2
5          10       Cao Thi Hanh  May Tinh         12    7.0


Các dòng có course_id là NaN trước đó đã được thay thế bằng mã môn học ngẫu nhiên từ bảng course.
Các dòng có course_id không tồn tại trong bảng course đã bị xóa.
Bảng kết quả chỉ còn lại các sinh viên có course_id hợp lệ và không còn giá trị NaN.

a. Tổng số sinh viên, điểm trung bình của từng lớp.



In [13]:
# Truy vấn tổng số sinh viên và điểm trung bình của từng lớp
query = '''
SELECT class, COUNT(student_id) AS total_students, ROUND(AVG(score), 2) AS avg_score
FROM student
GROUP BY class;
'''

df = pd.read_sql_query(query, conn)
print(df)

      class  total_students  avg_score
0   Kinh Te               3       8.53
1  May Tinh               2       6.85
2  Toan Tin               1       7.90


Kinh Te: 8.53 — Điểm trung bình cao nhất trong các lớp, cho thấy chất lượng học tập tốt.
May Tinh: 6.85 — Điểm trung bình thấp nhất, có thể cần cải thiện chất lượng học tập.
Toan Tin: 7.90 — Ở mức khá, có thể duy trì và cải thiện thêm. Lớp Toan Tin chỉ có 1 sinh viên, có thể ảnh hưởng đến tính đại diện của điểm trung bình.

b. Tổng số sinh viên, điểm trung bình của từng môn học.



In [14]:
# Truy vấn tổng số sinh viên và điểm trung bình của từng môn học
query = '''
SELECT course.course_name, COUNT(student.student_id) AS total_students, ROUND(AVG(student.score), 2) AS avg_score
FROM student
LEFT JOIN course ON student.course_id = course.id
GROUP BY course.course_name;
'''

df = pd.read_sql_query(query, conn)
print(df)

  course_name  total_students  avg_score
0   Giai tich               4        7.2
1    Thong ke               2        9.2


Thong ke là môn có kết quả tốt nhất với điểm trung bình cao.
Giai tich có thể cần cải thiện về cả số lượng sinh viên đăng ký và chất lượng học tập.
Tin hoc dù đông sinh viên nhưng điểm chưa cao, cần tập trung cải thiện.

c. Phân loại thi đua theo số điểm của từng môn học biết:

i. Điểm TB ≥ 9.0: Xuất sắc.
ii. 6.0 ≤ Điểm TB ≤ 8.9: Tốt.
iii. Điểm TB < 6.0: Kém.

In [15]:
# Truy vấn phân loại thi đua theo điểm trung bình của từng môn học
query = '''
SELECT course.course_name,
       ROUND(AVG(student.score), 2) AS avg_score,
       CASE
           WHEN ROUND(AVG(student.score), 2) >= 9.0 THEN 'Xuất sắc'
           WHEN ROUND(AVG(student.score), 2) BETWEEN 6.0 AND 8.9 THEN 'Tốt'
           ELSE 'Kém'
       END AS rating
FROM student
LEFT JOIN course ON student.course_id = course.id
GROUP BY course.course_name;
'''

df = pd.read_sql_query(query, conn)
print(df)

  course_name  avg_score    rating
0   Giai tich        7.2       Tốt
1    Thong ke        9.2  Xuất sắc


Môn Thong ke dẫn đầu với xếp loại Xuất sắc và điểm trung bình cao nhất.
Giai tich và Tin hoc xếp loại Tốt, cần có giải pháp nâng cao chất lượng.
Không có môn Kém là một tín hiệu tích cực cho chất lượng đào tạo hiện tại.

3. Hãy xếp hạng sinh viên thông qua:

a. Điểm số.

b. Điểm số theo lớp học.

c. Điểm số theo mã môn học.

và cho biết top 3 sinh viện đạt thứ hạng cao nhất, top 3 sinh viên đạt thứ hạng thấp nhất theo từng trường hợp trên.

a. Điểm số

In [16]:
# Truy vấn xếp hạng sinh viên theo điểm số
query = '''
SELECT student_id, name, class, course_id, score,
       RANK() OVER (ORDER BY score DESC) AS rank
FROM student
ORDER BY score DESC;
'''

df_rank = pd.read_sql_query(query, conn)
print(df_rank)

   student_id               name     class  course_id  score  rank
0           2       Tran Thi Lan   Kinh Te         34    9.2     1
1           7      Bui Tien Dung   Kinh Te         34    9.2     1
2           3       Pham Van Nam  Toan Tin         12    7.9     3
3           9     Duong Huu Phuc   Kinh Te         12    7.2     4
4          10       Cao Thi Hanh  May Tinh         12    7.0     5
5           1  Nguyen Minh Hoang  May Tinh         12    6.7     6


Lớp Kinh Te có thành tích nổi bật nhất với cả hạng 1 và hạng 4, thể hiện chất lượng học tập ổn định.
Lớp May Tinh có xu hướng điểm thấp hơn so với các lớp khác, cần xem xét cải thiện phương pháp giảng dạy.
Không có sinh viên dưới 6.5 điểm, thể hiện mặt bằng học lực khá tốt.

b. điểm số theo lớp học

In [17]:
# Truy vấn xếp hạng sinh viên theo điểm số trong từng lớp
query = '''
SELECT student_id, name, class, course_id, score,
       RANK() OVER (PARTITION BY class ORDER BY score DESC) AS class_rank
FROM student
ORDER BY class, class_rank;
'''

df_class_rank = pd.read_sql_query(query, conn)
print(df_class_rank)

   student_id               name     class  course_id  score  class_rank
0           2       Tran Thi Lan   Kinh Te         34    9.2           1
1           7      Bui Tien Dung   Kinh Te         34    9.2           1
2           9     Duong Huu Phuc   Kinh Te         12    7.2           3
3          10       Cao Thi Hanh  May Tinh         12    7.0           1
4           1  Nguyen Minh Hoang  May Tinh         12    6.7           2
5           3       Pham Van Nam  Toan Tin         12    7.9           1


Lớp Kinh Te có sự cạnh tranh cao với khoảng cách điểm lớn giữa các hạng.
Lớp May Tinh cần nâng cao chất lượng học tập do điểm số dẫn đầu chưa thực sự nổi bật.
Lớp Toan Tin có mặt bằng điểm khá nhưng thiếu sự cạnh tranh, cần thu hút thêm sinh viên

c. Điểm số theo mã môn học.



In [18]:
# Truy vấn xếp hạng sinh viên theo điểm số trong từng mã môn học
query = '''
SELECT student_id, name, class, course_id, score,
       RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS course_rank
FROM student
ORDER BY course_id, course_rank;
'''

df_course_rank = pd.read_sql_query(query, conn)
print(df_course_rank)

   student_id               name     class  course_id  score  course_rank
0           3       Pham Van Nam  Toan Tin         12    7.9            1
1           9     Duong Huu Phuc   Kinh Te         12    7.2            2
2          10       Cao Thi Hanh  May Tinh         12    7.0            3
3           1  Nguyen Minh Hoang  May Tinh         12    6.7            4
4           2       Tran Thi Lan   Kinh Te         34    9.2            1
5           7      Bui Tien Dung   Kinh Te         34    9.2            1


Môn Giai tich: Nguyễn Minh Hoàng là sinh viên duy nhất với 6.7 điểm (mức trung bình), cần cải thiện thêm.
Môn Tin hoc: Phạm Văn Nam dẫn đầu với 7.9 điểm; Dương Hữu Phúc và Cao Thị Hạnh lần lượt đạt 7.2 và 7.0 điểm. Điểm trung bình khá ổn, cần cố gắng hơn để đạt xuất sắc.
Môn Thong ke: Trần Thị Lan và Bùi Tiến Dũng cùng đạt 9.2 điểm (xuất sắc), có điểm trung bình cao nhất, thể hiện chất lượng học tập tốt.

Cho biết top 3 sinh viện đạt thứ hạng cao nhất, top 3 sinh viên đạt thứ hạng thấp nhất theo từng trường hợp trên.

Top 3 cao nhất & thấp nhất theo điểm số

In [19]:
# Truy vấn top 3 cao nhất và thấp nhất theo điểm số toàn bộ
query = '''
WITH ranked_students AS (
    SELECT student_id, name, class, course_id, score,
           RANK() OVER (ORDER BY score DESC) AS rank_desc,
           RANK() OVER (ORDER BY score ASC) AS rank_asc
    FROM student
)
SELECT * FROM ranked_students
WHERE rank_desc <= 3 OR rank_asc <= 3
ORDER BY rank_desc;
'''
df_top_all = pd.read_sql_query(query, conn)
print(" Top 3 cao nhất & thấp nhất theo điểm số toàn bộ:")
print(df_top_all)
print("\n")

 Top 3 cao nhất & thấp nhất theo điểm số toàn bộ:
   student_id               name     class  course_id  score  rank_desc  \
0           2       Tran Thi Lan   Kinh Te         34    9.2          1   
1           7      Bui Tien Dung   Kinh Te         34    9.2          1   
2           3       Pham Van Nam  Toan Tin         12    7.9          3   
3           9     Duong Huu Phuc   Kinh Te         12    7.2          4   
4          10       Cao Thi Hanh  May Tinh         12    7.0          5   
5           1  Nguyen Minh Hoang  May Tinh         12    6.7          6   

   rank_asc  
0         5  
1         5  
2         4  
3         3  
4         2  
5         1  




Top 3 cao nhất:

Trần Thị Lan và Bùi Tiến Dũng (9.2 điểm) — Hạng 1.
Phạm Văn Nam (7.9 điểm) — Hạng 3.
Top 3 thấp nhất:

Nguyễn Minh Hoàng (6.7 điểm) — Hạng 6 (thấp nhất).
Cao Thị Hạnh (7.0 điểm) — Hạng 5.
Dương Hữu Phúc (7.2 điểm) — Hạng 4.

Top 3 cao nhất & thấp nhất theo điểm số từng lớp

In [20]:
# Truy vấn top 3 cao nhất và thấp nhất theo điểm từng lớp
query = '''
WITH ranked_students AS (
    SELECT student_id, name, class, course_id, score,
           RANK() OVER (PARTITION BY class ORDER BY score DESC) AS class_rank_desc,
           RANK() OVER (PARTITION BY class ORDER BY score ASC) AS class_rank_asc
    FROM student
)
SELECT * FROM ranked_students
WHERE class_rank_desc <= 3 OR class_rank_asc <= 3
ORDER BY class, class_rank_desc;
'''

df_top_class = pd.read_sql_query(query, conn)
print(" Top 3 cao nhất & thấp nhất theo điểm số từng lớp:")
print(df_top_class)
print("\n")

 Top 3 cao nhất & thấp nhất theo điểm số từng lớp:
   student_id               name     class  course_id  score  class_rank_desc  \
0           2       Tran Thi Lan   Kinh Te         34    9.2                1   
1           7      Bui Tien Dung   Kinh Te         34    9.2                1   
2           9     Duong Huu Phuc   Kinh Te         12    7.2                3   
3          10       Cao Thi Hanh  May Tinh         12    7.0                1   
4           1  Nguyen Minh Hoang  May Tinh         12    6.7                2   
5           3       Pham Van Nam  Toan Tin         12    7.9                1   

   class_rank_asc  
0               2  
1               2  
2               1  
3               2  
4               1  
5               1  




Top 3 cao nhất & thấp nhất theo điểm số từng lớp

Lớp Kinh Tế:

Cao nhất: Trần Thị Lan và Bùi Tiến Dũng (9.2 điểm) — Đồng hạng 1.
Thấp nhất: Dương Hữu Phúc (7.2 điểm) — Hạng 3.
Lớp Máy Tính:

Cao nhất: Cao Thị Hạnh (7.0 điểm) — Hạng 1.
Thấp nhất: Nguyễn Minh Hoàng (6.7 điểm) — Hạng 2 (cũng là thấp nhất).
Lớp Toán Tin:

Cao nhất: Phạm Văn Nam (7.9 điểm) — Hạng 1.
Thấp nhất: Cũng là Phạm Văn Nam (7.9 điểm) — Hạng 1 (vì chỉ có 1 sinh viên).

Top 3 cao nhất & thấp nhất theo điểm số từng môn học



In [21]:
# Truy vấn top 3 cao nhất và thấp nhất theo điểm từng môn học
query = '''
WITH ranked_students AS (
    SELECT student_id, name, class, course_id, score,
           RANK() OVER (PARTITION BY course_id ORDER BY score DESC) AS course_rank_desc,
           RANK() OVER (PARTITION BY course_id ORDER BY score ASC) AS course_rank_asc
    FROM student
)
SELECT * FROM ranked_students
WHERE course_rank_desc <= 3 OR course_rank_asc <= 3
ORDER BY course_id, course_rank_desc;
'''

df_top_course = pd.read_sql_query(query, conn)
print(" Top 3 cao nhất & thấp nhất theo điểm số từng môn học:")
print(df_top_course)
print("\n")

 Top 3 cao nhất & thấp nhất theo điểm số từng môn học:
   student_id               name     class  course_id  score  \
0           3       Pham Van Nam  Toan Tin         12    7.9   
1           9     Duong Huu Phuc   Kinh Te         12    7.2   
2          10       Cao Thi Hanh  May Tinh         12    7.0   
3           1  Nguyen Minh Hoang  May Tinh         12    6.7   
4           2       Tran Thi Lan   Kinh Te         34    9.2   
5           7      Bui Tien Dung   Kinh Te         34    9.2   

   course_rank_desc  course_rank_asc  
0                 1                4  
1                 2                3  
2                 3                2  
3                 4                1  
4                 1                1  
5                 1                1  




Top 3 cao nhất & thấp nhất theo điểm số từng môn học

Môn Giải tích:

Cao nhất: Nguyễn Minh Hoàng (6.7 điểm) — Hạng 1.
Thấp nhất: Cũng là Nguyễn Minh Hoàng (6.7 điểm) — Hạng 1 vì chỉ có 1 sinh viên.
Môn Tin học:

Cao nhất: Phạm Văn Nam (7.9 điểm) — Hạng 1.
Thấp nhất: Cao Thị Hạnh (7.0 điểm) — Hạng 3.
Môn Thống kê:

Cao nhất: Trần Thị Lan và Bùi Tiến Dũng (9.2 điểm) — Đồng hạng 1.
Thấp nhất: Cũng là Trần Thị Lan và Bùi Tiến Dũng (9.2 điểm) — Đồng hạng 1 do điểm số bằng nhau.

4. Hãy bổ sung thêm một trường graduation_date có kiểu dữ liệu là DATETIME vào bảng student để xác định thời gian tốt nghiệp của từng bạn, trong đó thời gian tốt nghiệp được xác định bởi thời gian hiện tại cộng với số hạng tương ứng của bạn đó tính theo điểm số.

Thêm cột graduation_date vào bảng student



In [22]:
# Thêm cột graduation_date với kiểu DATETIME nếu chưa có
cursor.execute('''
ALTER TABLE student ADD COLUMN graduation_date DATETIME;
''')
conn.commit()

Cập nhật graduation_date dựa trên thứ hạng



In [23]:
# Cập nhật graduation_date dựa trên thứ hạng
cursor.execute('''
WITH ranked_students AS (
    SELECT student_id, RANK() OVER (ORDER BY score DESC) AS rank
    FROM student
)
UPDATE student
SET graduation_date = DATETIME('now', '+' || (
    SELECT rank FROM ranked_students WHERE ranked_students.student_id = student.student_id
) || ' months');
''')
conn.commit()

Kiểm tra lại bảng student với trường graduation_date



In [24]:
# Truy vấn lại để kiểm tra kết quả
query = '''
SELECT student_id, name, class, course_id, score, graduation_date
FROM student
ORDER BY score DESC;
'''

df_updated = pd.read_sql_query(query, conn)
print(" Bảng `student` sau khi thêm trường `graduation_date`:")
print(df_updated)

 Bảng `student` sau khi thêm trường `graduation_date`:
   student_id               name     class  course_id  score  \
0           2       Tran Thi Lan   Kinh Te         34    9.2   
1           7      Bui Tien Dung   Kinh Te         34    9.2   
2           3       Pham Van Nam  Toan Tin         12    7.9   
3           9     Duong Huu Phuc   Kinh Te         12    7.2   
4          10       Cao Thi Hanh  May Tinh         12    7.0   
5           1  Nguyen Minh Hoang  May Tinh         12    6.7   

       graduation_date  
0  2025-04-11 08:49:38  
1  2025-04-11 08:49:38  
2  2025-06-11 08:49:38  
3  2025-07-11 08:49:38  
4  2025-08-11 08:49:38  
5  2025-09-11 08:49:38  


Bảng student sau khi thêm trường graduation_date:

Trần Thị Lan và Bùi Tiến Dũng (Kinh Tế):

Điểm: 9.2 — Ngày tốt nghiệp: 08/04/2025.
Phạm Văn Nam (Toán Tin):

Điểm: 7.9 — Ngày tốt nghiệp: 08/06/2025.
Dương Hữu Phúc (Kinh Tế):

Điểm: 7.2 — Ngày tốt nghiệp: 08/07/2025.
Cao Thị Hạnh (Máy Tính):

Điểm: 7.0 — Ngày tốt nghiệp: 08/08/2025.
Nguyễn Minh Hoàng (Máy Tính):

Điểm: 6.7 — Ngày tốt nghiệp: 08/09/2025.
Kết quả cho thấy trường graduation_date đã được cập nhật thành công và hợp lý trên dữ liệu